In [ ]:
E:\QuantLake>sqlite3 chinook.db

# TASK-1

In [ ]:
#1. List each customer with total number of orders and total revenue
sqlite> SELECT
   ...>     c.CustomerId,
   ...>     c.FirstName || ' ' || c.LastName AS CustomerName,
   ...>     COUNT(i.InvoiceId) AS TotalOrders,
   ...>     SUM(i.Total) AS TotalRevenue
   ...> FROM
   ...>     customers c
   ...> JOIN
   ...>     invoices i ON c.CustomerId = i.CustomerId
   ...> GROUP BY
   ...>     c.CustomerId
   ...> ORDER BY
   ...>     TotalRevenue DESC;

In [ ]:
#2. Retrieve the top 5 highest spending customers with their country names
sqlite> SELECT
   ...>     c.CustomerId,
   ...>     c.FirstName || ' ' || c.LastName AS CustomerName,
   ...>     c.Country,
   ...>     SUM(i.Total) AS TotalSpent
   ...> FROM
   ...>     customers c
   ...> JOIN
   ...>     invoices i ON c.CustomerId = i.CustomerId
   ...> GROUP BY
   ...>     c.CustomerId
   ...> ORDER BY
   ...>     TotalSpent DESC
   ...> LIMIT 5;

In [ ]:
#3. List product categories (genres) and their total revenue using join
sqlite> SELECT
   ...>     g.Name AS Genre,
   ...>     SUM(ii.UnitPrice * ii.Quantity) AS TotalRevenue
   ...> FROM
   ...>     invoice_items ii
   ...> JOIN
   ...>     tracks t ON ii.TrackId = t.TrackId
   ...> JOIN
   ...>     genres g ON t.GenreId = g.GenreId
   ...> GROUP BY
   ...>     g.GenreId
   ...> ORDER BY
   ...>     TotalRevenue DESC;

# TASK-2

In [ ]:
# 1. For each customer, rank their orders by order amount
sqlite> SELECT
   ...>     CustomerId,
   ...>     InvoiceId,
   ...>     Total,
   ...>     ROW_NUMBER() OVER (PARTITION BY CustomerId ORDER BY Total DESC) AS OrderRank
   ...> FROM
   ...>     invoices;

In [ ]:
#2. Calculate running total of sales by month
sqlite> WITH MonthlySales AS (
(x1...>   SELECT
(x1...>     strftime('%Y-%m', InvoiceDate) AS Month,
(x1...>     SUM(Total) AS MonthlyTotal
(x1...>   FROM
(x1...>     invoices
(x1...>   GROUP BY
(x1...>     strftime('%Y-%m', InvoiceDate)
(x1...> )
   ...> SELECT
   ...>   Month,
   ...>   MonthlyTotal,
   ...>   SUM(MonthlyTotal) OVER (ORDER BY Month) AS RunningTotal
   ...> FROM
   ...>   MonthlySales;

In [ ]:
#3. Rank top 3 products per category (genre) by revenue
sqlite> WITH RankedTracks AS (
(x1...>   SELECT
(x1...>       g.Name AS Genre,
(x1...>       t.Name AS TrackName,
(x1...>       SUM(ii.UnitPrice * ii.Quantity) AS Revenue,
(x1...>       RANK() OVER (PARTITION BY g.GenreId ORDER BY SUM(ii.UnitPrice * ii.Quantity) DESC) AS RevenueRank
(x1...>   FROM
(x1...>       invoice_items ii
(x1...>   JOIN
(x1...>       tracks t ON ii.TrackId = t.TrackId
(x1...>   JOIN
(x1...>       genres g ON t.GenreId = g.GenreId
(x1...>   GROUP BY
(x1...>       g.GenreId, t.TrackId
(x1...> )
   ...> SELECT *
   ...> FROM RankedTracks
   ...> WHERE RevenueRank <= 3;

# TASK-3

In [ ]:
#1. CTE to find customers with average order value > ₹1000
sqlite> WITH CustomerAvgOrder AS (
(x1...>   SELECT
(x1...>     CustomerId,
(x1...>     AVG(Total) AS AvgOrderValue
(x1...>   FROM
(x1...>     invoices
(x1...>   GROUP BY
(x1...>     CustomerId
(x1...> )
   ...> SELECT
   ...>   c.CustomerId,
   ...>   c.FirstName || ' ' || c.LastName AS CustomerName,
   ...>   a.AvgOrderValue
   ...> FROM
   ...>   CustomerAvgOrder a
   ...> JOIN
   ...>   customers c ON c.CustomerId = a.CustomerId
   ...> WHERE
   ...>   a.AvgOrderValue > 1000;

In [ ]:
# 2. CTE to filter orders in the last 3 months, then aggregate revenue
sqlite> WITH RecentInvoices AS (
(x1...>   SELECT *
(x1...>   FROM invoices
(x1...>   WHERE InvoiceDate >= date('now', '-3 months')
(x1...> )
   ...> SELECT
   ...>   BillingCountry,
   ...>   COUNT(*) AS OrderCount,
   ...>   SUM(Total) AS TotalRevenue
   ...> FROM
   ...>   RecentInvoices
   ...> GROUP BY
   ...>   BillingCountry
   ...> ORDER BY
   ...>   TotalRevenue DESC;

# TASK-4

In [ ]:
#Step 1: Create Monthly Product-Wise Sales Trend
sqlite> WITH MonthlyProductSales AS (
(x1...>   SELECT
(x1...>     t.TrackId,
(x1...>     t.Name AS ProductName,
(x1...>     strftime('%Y-%m', i.InvoiceDate) AS Month,
(x1...>     SUM(ii.Quantity) AS MonthlySales
(x1...>   FROM
(x1...>     invoice_items ii
(x1...>   JOIN
(x1...>     invoices i ON ii.InvoiceId = i.InvoiceId
(x1...>   JOIN
(x1...>     tracks t ON ii.TrackId = t.TrackId
(x1...>   GROUP BY
(x1...>     t.TrackId, strftime('%Y-%m', i.InvoiceDate)
(x1...> )
   ...> SELECT * FROM MonthlyProductSales
   ...> ORDER BY TrackId, Month;

In [ ]:
# Step 2: Identify Products with Decreasing Sales in the Last 3 Monthssqlite> WITH MonthlyProductSales AS (
(x1...>   SELECT
(x1...>     t.TrackId,
(x1...>     t.Name AS ProductName,
(x1...>     strftime('%Y-%m', i.InvoiceDate) AS Month,
(x1...>     SUM(ii.Quantity) AS MonthlySales
(x1...>   FROM
(x1...>     invoice_items ii
(x1...>   JOIN
(x1...>     invoices i ON ii.InvoiceId = i.InvoiceId
(x1...>   JOIN
(x1...>     tracks t ON ii.TrackId = t.TrackId
(x1...>   GROUP BY
(x1...>     t.TrackId, strftime('%Y-%m', i.InvoiceDate)
(x1...> ),
   ...> RecentThreeMonths AS (
(x1...>   SELECT DISTINCT strftime('%Y-%m', InvoiceDate) AS Month
(x1...>   FROM invoices
(x1...>   ORDER BY Month DESC
(x1...>   LIMIT 3
(x1...> ),
   ...> FilteredSales AS (
(x1...>   SELECT mps.*
(x1...>   FROM MonthlyProductSales mps
(x1...>   JOIN RecentThreeMonths rtm ON mps.Month = rtm.Month
(x1...> ),
   ...> Ranked AS (
(x1...>   SELECT
(x1...>     *,
(x1...>     RANK() OVER (PARTITION BY TrackId ORDER BY Month DESC) AS MonthRank
(x1...>   FROM FilteredSales
(x1...> )
   ...> SELECT TrackId, ProductName
   ...> FROM Ranked
   ...> GROUP BY TrackId
   ...> HAVING COUNT(*) = 3 AND
   ...>        MAX(CASE WHEN MonthRank = 1 THEN MonthlySales END) >
   ...>        MAX(CASE WHEN MonthRank = 2 THEN MonthlySales END) AND
   ...>        MAX(CASE WHEN MonthRank = 2 THEN MonthlySales END) >
   ...>        MAX(CASE WHEN MonthRank = 3 THEN MonthlySales END);